In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the URL for scraping
url = 'https://sportsdata.usatoday.com/football/nfl/injuries'

# Send a GET request to the page
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [19]:
all_team_list = soup.find_all("li",class_="class-z9EpWUN")
final_listdf = []
for team in all_team_list:
    team_name = team.find("div", class_="class-Zx9z7TM").text
    team_players = team.find_all("div",class_="class-1oQrPjR")
    team_injury_status = team.find_all("div",class_='class-hxeToJi')
    team_injury_location = team.find_all("span",class_='class-Xpihkyk')
    i = 0
    for player in team_players:
        player_name = player.find_all("span")[0].text
        player_pos = player.find_all("span")[1].text
        player_injury_status = team_injury_status[i].text
        player_injury_location = team_injury_location[i*2].text
        player_injury_date = team_injury_location[i*2+1].text
        i+=1
        final_listdf.append([team_name, player_name, player_pos, player_injury_status, player_injury_location, player_injury_date])
df = pd.DataFrame(final_listdf, columns=['team_name', 'player_name', 'player_pos','player_injury_status','player_injury_location','player_injury_date'])


In [20]:
df

,team_name,player_name,player_pos,player_injury_status,player_injury_location,player_injury_date
0,Arizona Cardinals,Trey McBride,TE,Out,concussion,Sep 29
1,Arizona Cardinals,Khyiris Tonga,DT,Out,knee,Sep 29
2,Arizona Cardinals,Isaiah Adams,G,Questionable,thumb,Sep 29
3,Arizona Cardinals,Kelvin Beachum,T,Questionable,hamstring,Sep 29
4,Arizona Cardinals,Justin Jones,DT,Injured-Reserve,torn triceps,Sep 25
...,...,...,...,...,...,...
702,Washington Commanders,Jordan Magee,LB,Injured-Reserve (DFR),knee,Aug 27
703,Washington Commanders,Norell Pollard,DT,Injured-Reserve,undisclosed,Aug 27
704,Washington Commanders,Taylor Stallworth,DT,Injured-Reserve,hamstring,Aug 27
705,Washington Commanders,Keandre Jones,LB,Injured-Reserve,undisclosed,Aug 21


In [21]:
import requests

# API endpoint
url = "https://site.api.espn.com/apis/site/v2/sports/football/nfl/teams"

def get_teams():
    try:
        # Send a GET request to the API
        response = requests.get(url)
        response.raise_for_status()  # Check if request was successful
        
        # Parse JSON response
        data = response.json()
        
        # Extract teams information
        teams = data.get('sports', [])[0].get('leagues', [])[0].get('teams', [])
        
        # Loop through teams and print name and ID
        teams_info = []
        for team in teams:
            team_info = team.get('team', {})
            team_name = team_info.get('displayName')
            team_id = team_info.get('id')
            teams_info.append({'name': team_name, 'id': team_id})
            print(f"Team: {team_name}, ID: {team_id}")
        
        return teams_info
    
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

# Call the function
teams = get_teams()


Team: Arizona Cardinals, ID: 22
Team: Atlanta Falcons, ID: 1
Team: Baltimore Ravens, ID: 33
Team: Buffalo Bills, ID: 2
Team: Carolina Panthers, ID: 29
Team: Chicago Bears, ID: 3
Team: Cincinnati Bengals, ID: 4
Team: Cleveland Browns, ID: 5
Team: Dallas Cowboys, ID: 6
Team: Denver Broncos, ID: 7
Team: Detroit Lions, ID: 8
Team: Green Bay Packers, ID: 9
Team: Houston Texans, ID: 34
Team: Indianapolis Colts, ID: 11
Team: Jacksonville Jaguars, ID: 30
Team: Kansas City Chiefs, ID: 12
Team: Las Vegas Raiders, ID: 13
Team: Los Angeles Chargers, ID: 24
Team: Los Angeles Rams, ID: 14
Team: Miami Dolphins, ID: 15
Team: Minnesota Vikings, ID: 16
Team: New England Patriots, ID: 17
Team: New Orleans Saints, ID: 18
Team: New York Giants, ID: 19
Team: New York Jets, ID: 20
Team: Philadelphia Eagles, ID: 21
Team: Pittsburgh Steelers, ID: 23
Team: San Francisco 49ers, ID: 25
Team: Seattle Seahawks, ID: 26
Team: Tampa Bay Buccaneers, ID: 27
Team: Tennessee Titans, ID: 10
Team: Washington Commanders, ID:

In [22]:
import requests
import json

def get_team_injuries(team_id):
    url = f'https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/teams/{team_id}/injuries'
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            injury_data = response.json()
            return injury_data
        else:
            print(f"Error: Unable to fetch data. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example usage
if __name__ == "__main__":
    team_id = "22"  # Arizona
    injuries = get_team_injuries(team_id)
    
    if injuries:
        # Pretty print the JSON response
        # print(json.dumps(injuries, indent=4))
        for item in injuries['items']:
            response = requests.get(item['$ref'])
            injuryData = response.json()

            injuryShortComment = injuryData['shortComment']
            injuryLongComment = injuryData['longComment']
            
            athleteData = requests.get(injuryData["athlete"]['$ref']).json()
            athleteFirstName = athleteData["firstName"]
            athleteLastName = athleteData["lastName"]

            # print(athleteFirstName + " " + athleteLastName)
            print(injuryData)
    else:
        print("No data available.")


{'$ref': 'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/2024/athletes/15035/injuries/577371?lang=en&region=us', 'id': '577371', 'longComment': "Beachum missed last week's loss to the Lions, but he'll return to action in Week 4. He should be the Cardinals' starter at right tackle.", 'shortComment': "Beachum (hamstring) is active Sunday against the Commanders, Darren Urban of the team's official site reports.", 'status': 'Active', 'date': '2024-09-29T18:37Z', 'athlete': {'$ref': 'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/2024/athletes/15035?lang=en&region=us'}, 'team': {'$ref': 'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/2024/teams/22?lang=en&region=us'}, 'source': {'id': '1', 'description': 'basic/manual', 'state': 'basic'}, 'type': {'id': '0', 'name': 'INJURY_STATUS_ACTIVE', 'description': 'active', 'abbreviation': 'A'}}
{'$ref': 'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/2024

In [23]:
url = 'https://www.draftsharks.com/fantasy/injury-history/deandre-hopkins/6679'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
table_rows = soup.find("div",class_="injury-history-table-container").find("tbody").findAll("tr")
for row in table_rows:
    row_data = row.findAll("td")
    injury_date = row_data[0] # Date
    injury_league = row_data[1] # League
    print(injury_league)
    injury = row_data[2] # Injury
    injury_details = row_data[3] # Description


<td>NFL</td>
<td>NFL</td>
<td>NFL</td>
<td>NFL</td>
<td>NFL</td>
<td>NFL</td>
<td>NFL</td>
<td>NFL</td>
<td>NFL</td>
<td>NFL</td>
<td>NFL</td>
<td>NFL</td>
<td>NFL</td>
<td>NFL</td>
<td>NFL</td>
<td>NFL</td>
